# Дашборд для Яндекс.Дзен. 

 Анализ взаимодействия пользователей с карточками Яндекс.Дзен.
 
 Каждую карточку определяют её тема и источник (у него тоже есть тема). Примеры тем: «Красота и здоровье», «Россия», «Путешествия».
Пользователей системы характеризует возрастная категория. Скажем, «26-30» или «45+».

Есть три способа взаимодействия пользователей с системой:
- Карточка отображена для пользователя (show);
- Пользователь кликнул на карточку (click);
- Пользователь просмотрел статью карточки (view).

Каждую неделю возникают одни и те же вопросы:
- Сколько взаимодействий пользователей с карточками происходит в системе с разбивкой по темам карточек?
- Как много карточек генерируют источники с разными темами?
- Как соотносятся темы карточек и темы источников?

## Kраткое ТЗ:

Бизнес-задача: анализ взаимодействия пользователей с карточками Яндекс.Дзен;

Насколько часто предполагается пользоваться дашбордом: не реже, чем раз в неделю;

Кто будет основным пользователем дашборда: менеджеры по анализу контента;

Состав данных для дашборда:
- История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);
- Разбивка событий по темам источников;
- Таблица соответствия тем источников темам карточек;

По каким параметрам данные должны группироваться:
- Дата и время;
- Тема карточки;
- Тема источника;
- Возрастная группа;

Характер данных:
- История событий по темам карточек — абсолютные величины с разбивкой по минутам;
- Разбивка событий по темам источников — относительные величины (% событий);
- Соответствия тем источников темам карточек - абсолютные величины;
- Важность: все графики имеют равную важность;
Источники данных для дашборда: cырые данные о событиях взаимодействия пользователей с карточками (таблица log_raw);

База данных, в которой будут храниться агрегированные данные: дополнительные агрегированные таблицы в БД zen;

Частота обновления данных: один раз в сутки, в полночь по UTC;

Какие графики должны отображаться и в каком порядке, какие элементы управления должны быть на дашборде (макет дашборда):
https://drive.google.com/file/d/1RGUmWBwGRIvLi79uWdA7wG72r9xBnZK1/view?usp=sharing

# Подготовим файл с расширением csv для построения дашборда в Tableau Public

## Выгрузим данные из базы.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
!pip install psycopg2-binary

db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': '', # пароль
            'host': '            'port': , # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string)

In [2]:
query = '''SELECT * FROM dash_visits''' # sql запрос

In [3]:
dash_visits = pd.io.sql.read_sql(query, con = engine) # сохраняем результат выполнения в DataFrame

## Проверим данные

In [4]:
dash_visits.head(10)

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27
5,1040602,Деньги,Авто,18-25,2019-09-24 18:57:00,22
6,1040603,Деньги,Авто,18-25,2019-09-24 18:58:00,24
7,1040604,Деньги,Авто,18-25,2019-09-24 18:59:00,20
8,1040605,Деньги,Авто,18-25,2019-09-24 19:00:00,5
9,1040606,Деньги,Авто,26-30,2019-09-24 18:29:00,1


***Описание данных***

- record_id - номер записи;
- item_topic - тема статьи;
- source_topic - тема источника;
- age_segment - возрастная категория пользователя;
- dt - дата и время;
- visits - количество действий пользователей по отношению к карточке статьи.

In [5]:
dash_visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB


In [6]:
dash_visits.describe().T # посмотрим на цифры

,count,mean,std,min,25%,50%,75%,max
record_id,30745.0,1.055969e+06,8875.461350,1040597.0,1048283.0,1055969.0,1063655.0,1071341.0
visits,30745.0,1.008967e+01,19.727601,1.0,1.0,3.0,10.0,371.0


In [7]:
dash_visits.query('visits > 10') # данных со значением больше Q3

,record_id,item_topic,source_topic,age_segment,dt,visits
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27
5,1040602,Деньги,Авто,18-25,2019-09-24 18:57:00,22
6,1040603,Деньги,Авто,18-25,2019-09-24 18:58:00,24
7,1040604,Деньги,Авто,18-25,2019-09-24 18:59:00,20
...,...,...,...,...,...,...
30650,1071247,Юмор,Спорт,26-30,2019-09-24 18:59:00,11
30658,1071255,Юмор,Спорт,31-35,2019-09-24 18:58:00,11
30666,1071263,Юмор,Спорт,36-40,2019-09-24 18:56:00,15
30668,1071265,Юмор,Спорт,36-40,2019-09-24 18:58:00,15


In [8]:
dash_visits.duplicated().sum() # проверим на дубликаты

0

In [9]:
dash_visits[['item_topic', 'source_topic', 'age_segment', 'dt', 'visits']].duplicated().sum() # проверим на дубликаты без id

0

In [10]:
round((dash_visits.isna().mean()*100),2) # проверим на пропуски

record_id       0.0
item_topic      0.0
source_topic    0.0
age_segment     0.0
dt              0.0
visits          0.0
dtype: float64

In [11]:
round((dash_visits.isnull().mean()*100),2) # проверим на 0

record_id       0.0
item_topic      0.0
source_topic    0.0
age_segment     0.0
dt              0.0
visits          0.0
dtype: float64

In [12]:
print('Время записи данных между {} и {}'.format(dash_visits['dt'].min(), dash_visits['dt'].max()))

Время записи данных между 2019-09-24 18:28:00 и 2019-09-24 19:00:00


В ходе проверки данных пропусков и нулевых значенийне обнаружено. 

Типы данных в исходных столбцах не меняем.

Данные в таблице за 24 сентября 2019 года с 18:28:00 до 19:00:00.

## Экспортируем данные в csv-файл

In [14]:
dash_visits.to_csv('dash_visits.csv', sep='\t', encoding='utf-8', index=False)

Данные экспортированы в файл - 'dash_visits.csv'.